# Meerkat and alerta

In [23]:
import redis

In [24]:
r = redis.Redis(host="192.168.10.15", port=6379)

In [25]:
eve = r.lrange("suricata", 0, -1)

In [26]:
print(len(eve))

1840


In [27]:
import json
eve = [json.loads(e.decode()) for e in r.lrange("suricata", 0, -1) if e]
eve = [e for e in eve if e["event_type"] == "alert"]

In [28]:
print(eve)

[{'timestamp': '2019-02-06T16:49:24.984983+0000', 'flow_id': 1089646357698266, 'in_iface': 'enp0s3', 'event_type': 'alert', 'src_ip': '217.160.0.187', 'src_port': 80, 'dest_ip': '10.0.2.15', 'dest_port': 39324, 'proto': 'TCP', 'alert': {'action': 'allowed', 'gid': 1, 'signature_id': 2100498, 'rev': 7, 'signature': 'GPL ATTACK_RESPONSE id check returned root', 'category': 'Potentially Bad Traffic', 'severity': 2, 'metadata': {'updated_at': ['2010_09_23'], 'created_at': ['2010_09_23']}}, 'http': {'hostname': 'testmyids.com', 'url': '/', 'http_user_agent': 'curl/7.58.0', 'http_content_type': 'text/html', 'http_method': 'GET', 'protocol': 'HTTP/1.1', 'status': 200, 'length': 39}, 'app_proto': 'http', 'flow': {'pkts_toserver': 5, 'pkts_toclient': 4, 'bytes_toserver': 367, 'bytes_toclient': 532, 'start': '2019-02-06T16:49:24.906970+0000'}, 'host': 'CDMCS-rules'}, {'timestamp': '2019-02-06T16:49:24.984589+0000', 'flow_id': 1089646357698266, 'in_iface': 'enp0s3', 'event_type': 'alert', 'src_ip

In [29]:
import json
import requests

assets = { 
    "192.168.10.11": "singlehost",
    "192.168.10.15": "clusterbox"
}

for data in eve:
    
    if data["src_ip"] in assets:
        resource = data["src_ip"]
    elif data["dest_ip"] in assets:
        resource = data["dest_ip"]
    else:
        resource = data["dest_ip"]

    headers = { "Content-Type": "application/json" }
    alert = {
            "environment": "Production",
            "event": data["alert"]["signature"],
            "resource": resource,
            "text": "ALERT: {}".format(data["alert"]["signature"]),
            "service": [resource],
            "severity": "wat",
            "value": data["alert"]["severity"],
            "timeout": 60,
            }
    
    url = "http://192.168.10.15:8080/alert"
    resp = requests.post(url, data=json.dumps(alert), headers=headers)
    
    print(resp.json())

{'alert': {'attributes': {'ip': '192.168.10.15'}, 'correlate': [], 'createTime': '2019-02-06T16:53:04.636Z', 'customer': None, 'duplicateCount': 2, 'environment': 'Production', 'event': 'GPL ATTACK_RESPONSE id check returned root', 'group': 'Misc', 'history': [{'event': 'GPL ATTACK_RESPONSE id check returned root', 'href': 'http://192.168.10.15:8080/alert/64d222e1-4d9f-44d9-8593-638a5c43cb2d', 'id': '64d222e1-4d9f-44d9-8593-638a5c43cb2d', 'severity': 'wat', 'status': 'open', 'text': 'new alert', 'type': 'new', 'updateTime': '2019-02-06T16:53:04.636Z', 'value': '2'}, {'event': 'GPL ATTACK_RESPONSE id check returned root', 'href': 'http://192.168.10.15:8080/alert/79c87728-ae16-4cfa-88b6-e4522f0c91c4', 'id': '79c87728-ae16-4cfa-88b6-e4522f0c91c4', 'severity': 'wat', 'status': 'open', 'text': 'duplicate alert (with status change)', 'type': 'status', 'updateTime': '2019-02-06T16:56:53.819Z', 'value': '2'}], 'href': 'http://192.168.10.15:8080/alert/64d222e1-4d9f-44d9-8593-638a5c43cb2d', 'id'